# **Machine Learning Project**

In questa esercitazione metteremo assieme tutte le nozioni apprese dall'inizio del corso per risolvere un task specifico di machine learning.

### **Task: Semi-supervised classification**

Il task che vogliamo risolvere è un task di classificazione, caratterizzato però dal fatto che solo una piccola parte dei dati che disponiamo possiede le annotazioni (label). Questa condizione è nota come `Semi-supervised learning`. 

### **Dataset**

Il dataset che utilizzeremo sarà Fashion-MNIST, che contiene immagini di articoli di Zalando, composto da un training set di 60.000 campioni e test set con 10.000 campioni. Ogni campione è in scala di grigi e ha risoluzione 28x28. Il dataset è composto da 10 classi.


## **Pseudo-label**

Lo **pseudo-labeling** è una tecnica utilizzata nell'ambito del *semi-supervised learning*. L'idea di base è quella di generare etichette "artificiali" (pseudo-etichette) per i dati non etichettati (unlabeled, "U"), in modo da utilizzarle durante il training del modello. Per generare queste etichette ci sono diverse strategie: nel contesto di questa esercitazione utilizzeremo un algoritmo di clustering (k-means).

Ecco i passaggi generali del processo di pseudo-labeling:

1.  **Addestramento Iniziale**: Si addestra un algortimo di clustering sul set non etichettato (U) utilizzando un numero di cluster pari al numero di classi.
2.  **Predizione su Dati Etichettati**: Utilizziamo l'algortimo addestrato al punto 1 per clusterizzare i dati etichettati (L), assegnandoli quindi ai cluster che abbiamo trovato durante l' addestramento iniziale.
3.  **Mappare i cluster alle etichette**: Creiamo un mapping tra i cluster e le etichette, in modo da capire quale etichetta corrisponde allo specifico cluster. Per fare ciò assegniamo ad ogni cluster la vera label più frequente assegnata a quel cluster, sfruttando la funzione `mode`:

```Python
from scipy.stats import mode
import numpy as np

etichette_nel_cluster = np.array([0, 1, 1, 2, 1, 0, 1])

risultato_mode = mode(etichette_nel_cluster)

print(f"Oggetto ModeResult: {risultato_mode}") # Output: ModeResul(mode=1, count=4)
print(f"Etichetta più frequente (moda): {risultato_mode.mode}") # Output: (moda): 1

# etichetta più frequente come singolo numero:
etichetta_predominante = risultato_mode.mode
print(f"Etichetta predominante per questo cluster: {etichetta_predominante}") # Output: 1
```

4.  **Estrazione pseudo-label**: Alla fine, la classe più presente in un cluster diventa l' etichetta scelta per tutti i campioni assegnati a quel cluster. 


**Vantaggi**:
*   Permette di sfruttare la grande quantità di dati non etichettati, che altrimenti andrebbero sprecati.
*   Può migliorare significativamente le prestazioni del modello rispetto all'addestramento con i soli dati etichettati, specialmente quando questi ultimi sono scarsi.

In [2]:
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import mode # For majority voting
from sklearn.neural_network import MLPClassifier

In [3]:
# Nomi delle classi per Fashion-MNIST
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### `load_and_preprocess_data()`

In questa funzione dovrete:

* Scaricare il dataset.
* Riordinare casualmente i dati.
* Effettuare reshape.
* Scalare i valori dei pixel all' intervallo [0,1].
* Ridurre il numero di campioni a 10.000 per il train e 1.000 per il test.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set ridotto.
2. Le etichette di train ridotte.
3. Il test set ridotto.
4. Le etichette di test ridotte.

In [5]:
def load_and_preprocess_data():
    """Carica e pre-processa il dataset Fashion-MNIST."""
    np.random.seed(0)

    (x_train, y_train), (x_test,y_test) = fashion_mnist.load_data()

    indices = np.arange(x_train.shape[0]) 
    np.random.shuffle(indices)            
    x_train = x_train[indices]            
    y_train = y_train[indices]
    
    indices = np.arange(x_test.shape[0]) 
    np.random.shuffle(indices) 
    x_test = x_test[indices]
    y_test = y_test[indices]

    x_train = x_train.reshape(x_train.shape[0], -1)
    x_test = x_test.reshape(x_test.shape[0], -1)

    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    x_train_reduced = x_train[:10000]
    x_test_reduced = x_test[:1000]
    y_train_reduced = y_train[:10000]
    y_test_reduced = y_test[:1000]

    print("Dimensioni training: ", x_train_reduced.shape ,y_train_reduced.shape)
    print("Dimensioni test: ", x_test_reduced.shape, y_train_reduced.shape)
    
    return x_train_reduced,y_train_reduced,x_test_reduced,y_test_reduced

### `apply_pca_and_scale`

In questa funzione dovrete:

* Scalare il training set e il test set.
* Applicare PCA con un numero di componenti specificato come parametro della funzione (o, equivalentemente, con una frazione desiderata della varianza espressa).
* Stampare il numero di componenti.
* Stampare la varianza espressa.

La funzione dovrà ritornare nel seguente ordine:

1. Il training set trasformato con PCA.
2. Il test set trasformato con PCA.

In [8]:
def apply_pca_and_scale(x_train, x_test, n_components):
    """Applica StandardScaler e PCA."""

    standard_scaler = StandardScaler()
    x_train_std = standard_scaler.fit_transform(x_train)
    x_test_std = standard_scaler.transform(x_test)
    
    pca = PCA(n_components=n_components)
    x_train_pca = pca.fit_transform(x_train_std)
    x_test_pca = pca.transform(x_test_std)
    
    # Informazioni diagnostiche
    print(f"\nPCA applicata con {n_components} componenti")
    print(f"Varianza totale conservata: {np.sum(pca.explained_variance_ratio_):.3f}")
    print(f"Shape risultanti - Train: {x_train_pca.shape}, Test: {x_test_pca.shape}")
    
    return x_train_pca, x_test_pca

### `create_semi_supervised_split`

In questa funzione dovrete:

* Splittare il training set in due insiemi, etichettato (L) e non etichettato (U) utilizzando  `train_test_split` con:

`test_size`=`(1.0 - labeled_fraction)`

* Stampare la shape del set etichettato.
* Stampare la shape del set non etichettato.

La funzione deve ritornare nell seguente ordine:

1. Il set etichettato.
2. Le etichette del set etichettato.
3. Il set non etichettato.
4. Le etichette del set non etichettato. **N.B.** Queste etichette verranno utilizzate **SOLO** per valutare le pseudo-labels, non per l'addestramento.


In [9]:
def create_semi_supervised_split(x_train_pca, y_train, labeled_fraction):
    """Crea gli insiemi etichettato (L) e non etichettato (U)."""
    L, U, y_l, y_u = train_test_split(x_train_pca, y_train, test_size=(1.0 - labeled_fraction), stratify=y_train, random_state = 42)
    print(L.shape, U.shape)
    
    return L, y_l, U, y_u

### `get_pseudo_labels`

In questa funzione dovrete:

* Istanziare un algoritmo di clustering (ad esempio, k-means).
* Addestrare e predire i clustering sul set non etichettato.
* Predire i clustering del set etichettato.
* Mappare i cluster ad un etichetta, utilizzando per ogni cluster l'etichetta più presente, estraibile utilizzando la funzione `mode` presentata sopra.
* Generare un array `pseudo_labels` assegnando a ogni campione del set non etichettato l'etichetta corrispondente al cluster a cui è stato assegnato.

La funzione deve ritornare:

1. L' array `pseudo_labels`.

In [10]:
def get_pseudo_labels(x_unlabeled_pca, x_labeled_pca, y_labeled, n_clusters):

    # 1. Istanziare algoritmo di clustering
    clustering_model = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)

    # 2. Allenare il modello di clustering con i dati non etichettati e salvare le assegnazioni ai cluster
    clustering_model.fit(x_unlabeled_pca)
    cluster_assignments_unlabeled = clustering_model.labels_

    # 3. Calcolare l'assegnamento dei dati etichettati ai cluster
    cluster_assignments_labeled = clustering_model.predict(x_labeled_pca)

    # 4. Mappiamo i cluster alle label vere più frequenti
    cluster_to_true_label_map = {}
    for k_idx in range(n_clusters):
        # 4.1 Troviamo per ogni cluster le etichette vere dei campioni che vi appartengono.
        mask = (cluster_assignments_labeled == k_idx)
        labels_in_cluster = y_labeled[mask]

        # 4.2 Troviamo l'etichetta più frequente per quel cluster.
        if len(labels_in_cluster) > 0:
            most_common_label = mode(labels_in_cluster, keepdims=True).mode[0]
        else:
            most_common_label = np.nan

        # 4.3 Salviamo l'etichetta più frequente per quel cluster in cluster_to_true_label_map.
        cluster_to_true_label_map[k_idx] = most_common_label

    pseudo_labels_list = []
    for c_assign in cluster_assignments_unlabeled:
        if c_assign in cluster_to_true_label_map:
            # Assegnamo l'etichetta più frequente trovata per quel cluster
            pseudo_labels_list.append(cluster_to_true_label_map[c_assign])
        else:
            # Se il cluster non ha una mappatura, possiamo assegnare un'etichetta di default o ignorarlo
            pseudo_labels_list.append(np.nan)
    
    # 5. Convertiamo la lista in un array numpy
    pseudo_labels = np.array(pseudo_labels_list)
    
    return pseudo_labels

### `train_and_evaluate_classifier`

In questa funzione dovrete:

* Rimuovere eventuali campioni con etichette NaN (potrebbero provenire da pseudo labels non mappate).
* Istanziare il modello utilizzando `model_class` come oggetto e `classifier_args` come argomenti. Esempio:

```Python
model_class = MLPClassifier
classifier_args = {'max_iter': 200, 'hidden_layer_sizes': (100, 50)}
model = model_class(**classifier_args)

# Equivalente a:
model = MLPClassifier(max_iter=200, hidden_layer_sizes=(100, 50))

```

* Allenare il modello sul training set a cui sono stati rimossi i campioni con etichette NaN.
* Calcolare l' accuracy.
* Stampare il `title`, che consiste nel titolo dell' esperimento eseguito. Questo perchè tale funzione verrà riutilizzata diverse volte per più set di dati. Un titolo ci permetterà di identificare quali risultati stiamo producendo.
* Stampare l' accuracy.
* Stampare il classification report.

La funzione deve ritornare:

1. Il modello.


In [11]:
def train_and_evaluate_classifier(model_class, classifier_args, x_train, y_train, x_test, y_test, title, class_names_list):
    valid_indices_train = ~np.isnan(y_train)
    x_train = x_train[valid_indices_train]
    y_train = y_train[valid_indices_train]
    model = model_class(**classifier_args)
    model.fit(x_train,y_train)
    prediction = model.predict(x_test)
    accuracy = accuracy_score(y_test,prediction)

    print(title)
    print("accuracy: ", accuracy)
    print("Classification Report:\n", classification_report(y_test,prediction, target_names = class_names_list))
    
    return model

### `main`

In questa funzione dovrete:

* Utilizzare la funzione `load_and_preprocess_data` per caricare e pre-processare i dati.
* Utilizzare la funzione `apply_pca_and_scale` per applicare scaling e PCA.
* Utilizzare la funzione `create_semi_supervised_split` per dividere il train set in set etichettato e non etichettato.
* Utilizzare la funzione `get_pseudo_labels` per calcoalre le pseudo labels sul set non etichettato.
* Calcolare l' accuracy delle pseudo labels, cioè confrontarle con quelle vere in modo da vedere quanto sono accurate.
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati etichettati (L).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello solo sui dati non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello sui dati etichettati (L) più quelli non etichettati (U).
* Utilizzare la funzione `train_and_evaluate_classifier` per allenare e valutare il modello su tutto il dataset originale.

In [13]:
def main(classifier_class, classifier_args, n_components_pca, labeled_fraction, n_clusters):
    # 1. Caricamento e pre-processing dei dati
    x_train, y_train, x_test, y_test = load_and_preprocess_data()
    
    # 2. Applicazione PCA e scaling
    x_train_pca, x_test_pca = apply_pca_and_scale(x_train, x_test, n_components_pca)
    
    # 3. Creazione split semi-supervisionato
    L, y_l, U, y_u = create_semi_supervised_split(x_train_pca, y_train, labeled_fraction)
    
    # 4. Calcolo pseudo-labels
    pseudo_labels = get_pseudo_labels(U, L, y_l, n_clusters)
    
    # 5. Valutazione accuratezza pseudo-labels
    valid_pseudo_indices = ~np.isnan(pseudo_labels)
    pseudo_labels_valid = pseudo_labels[valid_pseudo_indices]
    U_valid = U[valid_pseudo_indices]
    y_u_valid = y_u[valid_pseudo_indices]
    
    if len(pseudo_labels_valid) > 0:
        pseudo_accuracy = accuracy_score(y_u_valid, pseudo_labels_valid)
        print(f"\nAccuracy delle pseudo-labels: {pseudo_accuracy:.3f}")
        print("Classification Report per pseudo-labels:\n", 
              classification_report(y_u_valid, pseudo_labels_valid, target_names=class_names))
    else:
        print("\nAttenzione: nessuna pseudo-label valida generata")
    
    # 6. Addestramento e valutazione modelli
    print("\n" + "="*50)
    print("Modello addestrato solo sui dati etichettati (L)")
    model_L = train_and_evaluate_classifier(
        classifier_class, classifier_args, 
        L, y_l, 
        x_test_pca, y_test,
        "Risultati con solo dati etichettati (L):",
        class_names
    )
    
    print("\n" + "="*50)
    print("Modello addestrato solo sui dati non etichettati con pseudo-labels (U)")
    model_U = train_and_evaluate_classifier(
        classifier_class, classifier_args, 
        U_valid, pseudo_labels_valid, 
        x_test_pca, y_test,
        "Risultati con solo dati non etichettati (U) e pseudo-labels:",
        class_names
    )
    
    print("\n" + "="*50)
    print("Modello addestrato su dati etichettati (L) + non etichettati (U) con pseudo-labels")
    # Combiniamo L e U_valid
    L_U_combined = np.vstack([L, U_valid])
    L_U_labels = np.concatenate([y_l, pseudo_labels_valid])
    model_LU = train_and_evaluate_classifier(
        classifier_class, classifier_args, 
        L_U_combined, L_U_labels, 
        x_test_pca, y_test,
        "Risultati con dati etichettati (L) + non etichettati (U) e pseudo-labels:",
        class_names
    )
    
    print("\n" + "="*50)
    print("Modello addestrato su tutto il dataset originale")
    model_full = train_and_evaluate_classifier(
        classifier_class, classifier_args, 
        x_train_pca, y_train, 
        x_test_pca, y_test,
        "Risultati con tutto il dataset originale:",
        class_names
    )
    
    return {
        'model_L': model_L,
        'model_U': model_U,
        'model_LU': model_LU,
        'model_full': model_full,
        'pseudo_accuracy': pseudo_accuracy if len(pseudo_labels_valid) > 0 else None
    }

### **Utilizzare la funzione `main`**

Specifichiamo adesso un set di parametri richiesti dalla funzione main e utilizziamola. Nello specifico la funzione main ha bisogno di:

* `classifier_class`: quale classificatore utilizzare, ad esempio `'MLPClassifier'`, `'LogisticRegression'` o altri visti in precedenza.
* `classifier_args`: un dizionario contenente i parametri del classificatore scelto, ad esempio un `MLPClassifier` necessiterà del parametro `hidden_layer_sizes`. Dipendentemente da quale classificatore scegliete dovrete creare il dizionario.
* `n_components_pca`: numero di componenti di PCA che vogliamo utilizzare. Se specifichiamo un valore compreso in [0, 1] questo verrà considerato come la percentuale di varianza che vogliamo mentenere.
* `labeled_fraction`: percentuale di dati da usare come insieme etichettato. Si consiglia il valore 0.002 corrispondente allo 0.2%, cioè 16 immagini su 8000.
* `n_clusters`: numero di cluster da utilizzare, nel nostro caso vogliamo che ci sia un cluster per ogni classe, quindi 10.

Infine utilizziamo la funzione main.

In [14]:
# Parametri
CLASSIFIER_CLASS = MLPClassifier  # Modello da usare, ad esempio LogisticRegression o SVC
CLASSIFIER_ARGS = {
    'max_iter': 20,
    'hidden_layer_sizes': (200,200)  # Aumenta il numero di iterazioni per la convergenza
}
N_COMPONENTS_PCA = 0.95  # Mantiene il 95% della varianza spiegata, o un numero fisso es. 50
LABELED_FRACTION = 0.002   # Frazione di dati da usare come insieme etichettato L
N_CLUSTERS = 10          # Fashion-MNIST ha 10 classi

In [15]:
main(
    CLASSIFIER_CLASS,
    CLASSIFIER_ARGS,
    N_COMPONENTS_PCA,
    LABELED_FRACTION,
    N_CLUSTERS
)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Dimensioni training:  (10000, 784) (10000,)
Dimensioni test:  (1000, 784) (10000,)

PCA applicata con 0.95 componenti
Varianza totale conservata: 0.950
Shape risultanti - Train: (10000, 245), Test: (1000, 245)
(20, 245) (9980, 245)

Accuracy delle pseudo-labels: 0.461
Classification Report per pseudo-labels:
               precision    recall  f1-score   support

 T-shirt/top       0.46      0.60      0.52       964
     Trouser       0.59      0.90      0.71       999
    Pullover       0.38      0.62      0.47      1005
       Dress       0.10      0.13      0.11      1022
        Coat       0.00      0.00      0.00      1007
      Sandal       0.38      0.89      0.53      1019
       Shirt       0.00      0.00      0.00       961
     Sneaker       0.00      0.00      0.00       957
         Bag  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_

Risultati con solo dati non etichettati (U) e pseudo-labels:
accuracy:  0.424
Classification Report:
               precision    recall  f1-score   support

 T-shirt/top       0.37      0.53      0.43        85
     Trouser       0.61      0.91      0.73        91
    Pullover       0.35      0.58      0.44       109
       Dress       0.11      0.17      0.13       102
        Coat       0.00      0.00      0.00       112
      Sandal       0.37      0.84      0.51       104
       Shirt       0.00      0.00      0.00        95
     Sneaker       0.00      0.00      0.00       101
         Bag       0.98      0.42      0.59       113
  Ankle boot       0.64      0.92      0.75        88

    accuracy                           0.42      1000
   macro avg       0.34      0.44      0.36      1000
weighted avg       0.34      0.42      0.35      1000


Modello addestrato su dati etichettati (L) + non etichettati (U) con pseudo-labels


/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and b

Risultati con dati etichettati (L) + non etichettati (U) e pseudo-labels:
accuracy:  0.428
Classification Report:
               precision    recall  f1-score   support

 T-shirt/top       0.36      0.53      0.43        85
     Trouser       0.60      0.91      0.72        91
    Pullover       0.36      0.59      0.45       109
       Dress       0.13      0.20      0.15       102
        Coat       0.00      0.00      0.00       112
      Sandal       0.38      0.84      0.52       104
       Shirt       0.00      0.00      0.00        95
     Sneaker       0.00      0.00      0.00       101
         Bag       0.98      0.43      0.60       113
  Ankle boot       0.66      0.91      0.77        88

    accuracy                           0.43      1000
   macro avg       0.35      0.44      0.36      1000
weighted avg       0.35      0.43      0.36      1000


Modello addestrato su tutto il dataset originale
Risultati con tutto il dataset originale:
accuracy:  0.851
Classification Re

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  warnings.warn(


{'model_L': MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=20),
 'model_U': MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=20),
 'model_LU': MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=20),
 'model_full': MLPClassifier(hidden_layer_sizes=(200, 200), max_iter=20),
 'pseudo_accuracy': 0.4607105538140021}